In [2]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

In [61]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

In [226]:
data = pd.read_csv('New Folder/hotel_bookings.csv')
data.head(3)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02


In [227]:
data.groupby("is_canceled")["reservation_status"].value_counts()

is_canceled  reservation_status
0            Check-Out             75166
1            Canceled              43017
             No-Show                1207
Name: reservation_status, dtype: int64

In [223]:
print("# of NaN in each columns:", df.isnull().sum(), sep='\n')

# of NaN in each columns:
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                          

In [16]:
def perc_mv(x, y):
    perc = y.isnull().sum() / len(x) * 100
    return perc

print('Missing value ratios:\nCompany: {}\nAgent: {}\nCountry: {}'.format(perc_mv(df, df['company']),
                                                                                   perc_mv(df, df['agent']),
                                                                                   perc_mv(df, df['country'])))

Missing value ratios:
Company: 94.30689337465449
Agent: 13.686238378423655
Country: 0.40874445095904177


In [228]:
data["agent"].value_counts().count()

333

In [229]:
# company is dropped
data = data.drop(['company'], axis = 1)

In [230]:
# We have also 4 missing values in children column. If there is no information about children, In my opinion those customers do not have any children.
data['children'] = data['children'].fillna(0)

In [231]:
# I wanted to label them manually. I will do the rest with get.dummies or label_encoder.
data['hotel'] = data['hotel'].map({'Resort Hotel':0, 'City Hotel':1})

data['arrival_date_month'] = data['arrival_date_month'].map({'January':1, 'February': 2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7,
                                                            'August':8, 'September':9, 'October':10, 'November':11, 'December':12})

In [232]:
# Lets copy data to check the correlation between variables. 
cor_data = data.copy()

In [233]:
cor_data = cor_data.drop(columns = ['reservation_status_date','reservation_status','agent'], axis = 1)

In [234]:
#Lets delete the NA rows of country column
indices = cor_data.loc[pd.isna(cor_data["country"]), :].index 
cor_data = cor_data.drop(cor_data.index[indices])   
cor_data.isnull().sum()

#There is no missing value in the data

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

In [235]:
#one-hot-encoding
df1 = cor_data.copy()
df1 = pd.get_dummies(data = df1, columns = ['meal', 'market_segment', 'distribution_channel',
                                            'reserved_room_type', 'assigned_room_type', 'customer_type','deposit_type'])

In [236]:
le = LabelEncoder()
df1['country'] = le.fit_transform(df1['country']) 

# There are more than 300 classes, so I wanted to use label encoder on this feature.

0

In [237]:
df1.corr()["is_canceled"].sort_values()

deposit_type_No Deposit       -0.477760
total_of_special_requests     -0.235595
required_car_parking_spaces   -0.194801
market_segment_Direct         -0.153629
distribution_channel_Direct   -0.150912
                                 ...   
market_segment_Groups          0.221511
country                        0.270254
lead_time                      0.291940
deposit_type_Non Refund        0.481318
is_canceled                    1.000000
Name: is_canceled, Length: 68, dtype: float64

In [238]:
#drop the row with negative values
df1 = df1.drop(df1.index[14530])

In [241]:
df1['is_canceled'] = df1['is_canceled'].replace({0:'A',1:'B'})
df1

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable
0,0,A,342,2015,7,27,1,0,0,2,0.0,0,135,0,0,0,3,0,0.00,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,0,A,737,2015,7,27,1,0,0,2,0.0,0,135,0,0,0,4,0,0.00,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,0,A,7,2015,7,27,1,0,1,1,0.0,0,59,0,0,0,0,0,75.00,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,0,A,13,2015,7,27,1,0,1,1,0.0,0,59,0,0,0,0,0,75.00,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,0,A,14,2015,7,27,1,0,2,2,0.0,0,59,0,0,0,0,0,98.00,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,1,A,23,2017,8,35,30,2,5,2,0.0,0,15,0,0,0,0,0,96.14,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
119386,1,A,102,2017,8,35,31,2,5,3,0.0,0,56,0,0,0,0,0,225.43,0,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0
119387,1,A,34,2017,8,35,31,2,5,2,0.0,0,43,0,0,0,0,0,157.71,0,4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
119388,1,A,109,2017,8,35,31,2,5,2,0.0,0,59,0,0,0,0,0,104.40,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [244]:
#Main Data

df1.to_csv('Main.csv', index=False, encoding='utf-8')

In [1]:
#REMOVE constant,quasi constant and duplicate features 
from sklearn.feature_selection import VarianceThreshold,mutual_info_classif
from sklearn.feature_selection import SelectKBest , SelectPercentile 


In [3]:
df1 = pd.read_csv('Main.csv')

In [4]:
y = df1["is_canceled"]
X = df1.drop(["is_canceled"], axis=1).values


In [7]:
df1.shape
X.shape
df1

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable
0,0,A,342,2015,7,27,1,0,0,2,...,0,0,0,0,0,1,0,1,0,0
1,0,A,737,2015,7,27,1,0,0,2,...,0,0,0,0,0,1,0,1,0,0
2,0,A,7,2015,7,27,1,0,1,1,...,0,0,0,0,0,1,0,1,0,0
3,0,A,13,2015,7,27,1,0,1,1,...,0,0,0,0,0,1,0,1,0,0
4,0,A,14,2015,7,27,1,0,2,2,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118896,1,A,23,2017,8,35,30,2,5,2,...,0,0,0,0,0,1,0,1,0,0
118897,1,A,102,2017,8,35,31,2,5,3,...,0,0,0,0,0,1,0,1,0,0
118898,1,A,34,2017,8,35,31,2,5,2,...,0,0,0,0,0,1,0,1,0,0
118899,1,A,109,2017,8,35,31,2,5,2,...,0,0,0,0,0,1,0,1,0,0


In [251]:
mi = mutual_info_classif(X,y)

In [252]:
mi = pd.Series(mi) 
mi.sort_values(ascending=False) 
#mi.index = x_train_unique.columns 

64    0.138829
65    0.131084
1     0.081290
17    0.080637
11    0.071116
        ...   
24    0.000000
37    0.000000
58    0.000000
21    0.000000
23    0.000000
Length: 67, dtype: float64

In [253]:
X_mi = pd.DataFrame(X)
select_feature = SelectKBest(mutual_info_classif,k=5).fit(X_mi,y)

In [256]:
selected_features_df = pd.DataFrame({'Feature':list(X_mi.columns), 
                                     'Scores':select_feature.scores_ }) 
selected_features_df= selected_features_df.sort_values(by='Scores',ascending = False) 

In [257]:
selected_features_df.to_csv('feature.csv', index=False, encoding='utf-8')

In [178]:
X = pd.DataFrame(X)

,Feature,Scores
16,16,1.401827e+07
1,1,1.110343e+06
11,11,1.863945e+05
17,17,4.625114e+04
66,66,2.416924e+04
...,...,...
56,56,8.100542e-01
45,45,2.300371e-01
47,47,3.719568e-02
24,24,1.131116e-02


In [301]:
#Z-Score 
from scipy.stats import zscore
#numeric_cols = df1.select_dtypes(include=[np.number]).columns
cancel = df1['is_canceled']
temp1= df1.drop(['is_canceled'],axis=1)
z = temp1.apply(zscore)
z['is_canceled'] = cancel
z

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,is_canceled
0,-1.415249,2.223447,-1.636256,0.144813,-0.012279,-1.685664,-0.932435,-1.316803,0.244733,-0.261055,-0.081617,0.940313,-0.181822,-0.10302,-0.08865,4.257097,-0.132199,-2.020395,-0.253446,-0.721241,0.542483,-0.0822,-0.371710,-0.313466,-0.099474,-0.04469,-0.078813,-0.211934,2.924218,-0.447053,-0.504986,-0.949988,-0.004101,-0.240300,2.685193,-0.040322,-2.148539,-0.006485,-1.603276,-0.097427,11.256696,-0.438467,-0.240417,-0.157834,-0.133533,-0.071276,-0.007104,-0.004101,-1.280631,-0.13612,7.036348,-0.518151,-0.263836,-0.180013,-0.147715,-0.077341,-0.054878,-0.048498,-0.0029,-0.004101,-0.188408,-0.069405,0.577373,-0.517041,0.376108,-0.373744,-0.036937,A
1,-1.415249,5.918392,-1.636256,0.144813,-0.012279,-1.685664,-0.932435,-1.316803,0.244733,-0.261055,-0.081617,0.940313,-0.181822,-0.10302,-0.08865,5.789066,-0.132199,-2.020395,-0.253446,-0.721241,0.542483,-0.0822,-0.371710,-0.313466,-0.099474,-0.04469,-0.078813,-0.211934,2.924218,-0.447053,-0.504986,-0.949988,-0.004101,-0.240300,2.685193,-0.040322,-2.148539,-0.006485,-1.603276,-0.097427,11.256696,-0.438467,-0.240417,-0.157834,-0.133533,-0.071276,-0.007104,-0.004101,-1.280631,-0.13612,7.036348,-0.518151,-0.263836,-0.180013,-0.147715,-0.077341,-0.054878,-0.048498,-0.0029,-0.004101,-0.188408,-0.069405,0.577373,-0.517041,0.376108,-0.373744,-0.036937,A
2,-1.415249,-0.910241,-1.636256,0.144813,-0.012279,-1.685664,-0.932435,-0.790525,-1.483649,-0.261055,-0.081617,-0.760816,-0.181822,-0.10302,-0.08865,-0.338808,-0.132199,-0.534836,-0.253446,-0.721241,0.542483,-0.0822,-0.371710,-0.313466,-0.099474,-0.04469,-0.078813,-0.211934,2.924218,-0.447053,-0.504986,-0.949988,-0.004101,-0.240300,2.685193,-0.040322,-2.148539,-0.006485,0.623723,-0.097427,-0.088836,-0.438467,-0.240417,-0.157834,-0.133533,-0.071276,-0.007104,-0.004101,-1.280631,-0.13612,7.036348,-0.518151,-0.263836,-0.180013,-0.147715,-0.077341,-0.054878,-0.048498,-0.0029,-0.004101,-0.188408,-0.069405,0.577373,-0.517041,0.376108,-0.373744,-0.036937,A
3,-1.415249,-0.854115,-1.636256,0.144813,-0.012279,-1.685664,-0.932435,-0.790525,-1.483649,-0.261055,-0.081617,-0.760816,-0.181822,-0.10302,-0.08865,-0.338808,-0.132199,-0.534836,-0.253446,-0.721241,0.542483,-0.0822,-0.371710,-0.313466,-0.099474,-0.04469,-0.078813,4.718447,-0.341972,-0.447053,-0.504986,-0.949988,-0.004101,4.161469,-0.372413,-0.040322,-2.148539,-0.006485,0.623723,-0.097427,-0.088836,-0.438467,-0.240417,-0.157834,-0.133533,-0.071276,-0.007104,-0.004101,0.780865,-0.13612,-0.142119,-0.518151,-0.263836,-0.180013,-0.147715,-0.077341,-0.054878,-0.048498,-0.0029,-0.004101,-0.188408,-0.069405,0.577373,-0.517041,0.376108,-0.373744,-0.03

In [302]:
z.isnull().sum()

hotel                            0
lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
                                ..
customer_type_Transient-Party    0
deposit_type_No Deposit          0
deposit_type_Non Refund          0
deposit_type_Refundable          0
is_canceled                      0
Length: 68, dtype: int64

In [246]:
z.to_csv('z_score.csv', index=False, encoding='utf-8')

In [269]:
y = z["is_canceled"]
X = z.drop(["is_canceled"], axis=1).values


In [270]:
mi2 = mutual_info_classif(X,y)

In [271]:
mi2 = pd.Series(mi2) 
mi2.sort_values(ascending=False) 
#mi.index = x_train_unique.columns 

65    0.135674
64    0.130024
1     0.082197
17    0.079966
11    0.069944
        ...   
46    0.000000
40    0.000000
25    0.000000
43    0.000000
45    0.000000
Length: 67, dtype: float64

In [272]:
X_mi2 = pd.DataFrame(X)
select_feature = SelectKBest(mutual_info_classif,k=5).fit(X_mi2,y)

In [273]:
selected_features_df = pd.DataFrame({'Feature':list(X_mi2.columns), 
                                     'Scores':select_feature.scores_ }) 
selected_features_df= selected_features_df.sort_values(by='Scores',ascending = False) 

In [274]:
selected_features_df.to_csv('feature2.csv', index=False, encoding='utf-8')

In [276]:
z.isnull().sum()

hotel                            0
lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
                                ..
customer_type_Transient-Party    0
deposit_type_No Deposit          0
deposit_type_Non Refund          0
deposit_type_Refundable          0
is_canceled                      0
Length: 68, dtype: int64

In [300]:
#Minmax Transformation
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

target = df1['is_canceled']
temp2= df1.drop(['is_canceled'],axis=1)
col = temp2.columns
min_max = scaler.fit_transform(temp2) 
mm = pd.DataFrame(min_max,columns = col) 
mm['is_canceled'] = target 
mm

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,is_canceled
0,0.0,0.464043,0.0,0.545455,0.500000,0.000000,0.000,0.000000,0.036364,0.0,0.0,0.767045,0.0,0.0,0.0,0.142857,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
1,0.0,1.000000,0.0,0.545455,0.500000,0.000000,0.000,0.000000,0.036364,0.0,0.0,0.767045,0.0,0.0,0.0,0.190476,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
2,0.0,0.009498,0.0,0.545455,0.500000,0.000000,0.000,0.024390,0.018182,0.0,0.0,0.335227,0.0,0.0,0.0,0.000000,0.0,0.013889,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
3,0.0,0.017639,0.0,0.545455,0.500000,0.000000,0.000,0.024390,0.018182,0.0,0.0,0.335227,0.0,0.0,0.0,0.000000,0.0,0.013889,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
4,0.0,0.018996,0.0,0.545455,0.500000,0.000000,0.000,0.048780,0.036364,0.0,0.0,0.335227,0.0,0.0,0.0,0.000000,0.0,0.018148,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118896,1.0,0.031208,1.0,0.636364,0.653846,0.966667,0.125,0.121951,0.036364,0.0,0.0,0.085227,0.0,0.0,0.0,0.000000,0.0,0.017804,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
118897,1.0,0.138399,1.0,0.636364,0.653846,1.000000,0.125,0.121951,0.054545,0.0,0.0,0.318182,0.0,0.0,0.0,0.000000,0.0,0.041746,0.0,0.4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
118898,1.0,0.046133,1.0,0.636364,0.653846,1.000000,0.125,0.121951,0.036364,0.0,0.0,0.244318,0.0,0.0,

In [248]:
mm.to_csv('min_max.csv', index=False, encoding='utf-8')

In [3]:
mm1 = pd.read_csv('min_max.csv')

In [5]:
mm1.isnull().sum()


hotel                            0
lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
                                ..
customer_type_Transient-Party    0
deposit_type_No Deposit          0
deposit_type_Non Refund          0
deposit_type_Refundable          0
is_canceled                      0
Length: 68, dtype: int64

In [6]:
y = mm1["is_canceled"]
X = mm1.drop(["is_canceled"], axis=1).values


In [9]:
mi3 = mutual_info_classif(X,y)

In [ ]:
mi3 = pd.Series(mi3) 
mi3.sort_values(ascending=False) 
#mi.index = x_train_unique.columns 

In [10]:
X_mi3 = pd.DataFrame(X)
select_feature = SelectKBest(mutual_info_classif,k=5).fit(X_mi3,y)

In [11]:
selected_features_df = pd.DataFrame({'Feature':list(X_mi3.columns), 
                                     'Scores':select_feature.scores_ }) 
selected_features_df= selected_features_df.sort_values(by='Scores',ascending = False) 

In [12]:
selected_features_df.to_csv('feature3.csv', index=False, encoding='utf-8')

In [249]:
cancel = df1['is_canceled']
temp2 = df1.drop(['is_canceled'],axis=1)
sq_root = temp2.apply(np.sqrt)
sq_root['is_canceled'] = cancel
sq_root

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,is_canceled
0,0.0,18.493242,44.888751,2.645751,5.196152,1.000000,0.000000,0.000000,1.414214,0.0,0.0,11.618950,0.0,0.0,0.0,1.732051,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
1,0.0,27.147744,44.888751,2.645751,5.196152,1.000000,0.000000,0.000000,1.414214,0.0,0.0,11.618950,0.0,0.0,0.0,2.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
2,0.0,2.645751,44.888751,2.645751,5.196152,1.000000,0.000000,1.000000,1.000000,0.0,0.0,7.681146,0.0,0.0,0.0,0.000000,0.0,8.660254,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
3,0.0,3.605551,44.888751,2.645751,5.196152,1.000000,0.000000,1.000000,1.000000,0.0,0.0,7.681146,0.0,0.0,0.0,0.000000,0.0,8.660254,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
4,0.0,3.741657,44.888751,2.645751,5.196152,1.000000,0.000000,1.414214,1.414214,0.0,0.0,7.681146,0.0,0.0,0.0,0.000000,0.0,9.899495,0.0,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,1.0,4.795832,44.911023,2.828427,5.916080,5.477226,1.414214,2.236068,1.414214,0.0,0.0,3.872983,0.0,0.0,0.0,0.000000,0.0,9.805101,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,A
119386,1.0,10.099505,44.911023,2.828427,5.916080,5.567764,1.414214,2.236068,1.732051,0.0,0.0,7.483315,0.0,0.0,0.0,0.000000,0.0,15.014326,0.0,1.414214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.

In [258]:
y = sq_root["is_canceled"]
X = sq_root.drop(["is_canceled"], axis=1).values


In [250]:
sq_root.to_csv('sq_root.csv', index=False, encoding='utf-8')

In [259]:
mi4 = mutual_info_classif(X,y)

In [260]:
mi4 = pd.Series(mi4) 
mi4.sort_values(ascending=False) 
#mi.index = x_train_unique.columns 

64    0.141544
65    0.129163
1     0.081114
17    0.078278
11    0.070219
        ...   
46    0.000000
56    0.000000
55    0.000000
23    0.000000
66    0.000000
Length: 67, dtype: float64

In [261]:
X_mi4 = pd.DataFrame(X)
select_feature = SelectKBest(mutual_info_classif,k=5).fit(X_mi4,y)

In [262]:
selected_features_df = pd.DataFrame({'Feature':list(X_mi4.columns), 
                                     'Scores':select_feature.scores_ }) 
selected_features_df= selected_features_df.sort_values(by='Scores',ascending = False) 

In [263]:
selected_features_df.to_csv('feature4.csv', index=False, encoding='utf-8')

In [161]:
from scipy.stats import boxcox 
for column in df1.columns: 
    posdata = df1[df1[column] > 0][column] 
    bcdata, lam = boxcox(posdata) 
    bc = np.empty_like(df1[column]) 
    bc[df1[column] > 0] = bcdata 
    bc[df1[column] == 0] = -1/lam 

ValueError: Data must not be constant.

In [213]:
#####  SVM  ##############
from sklearn import svm,metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB  
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression 
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import f1_score 
from imblearn.metrics import geometric_mean_score 
from imblearn.metrics import sensitivity_score 
from imblearn.metrics import specificity_score 
from sklearn.preprocessing import label_binarize 
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import cross_val_predict 



Using TensorFlow backend.


In [243]:
gb_clf = GradientBoostingClassifier(n_estimators=100,  max_features=2, max_depth=2, random_state=0)
y_pred = cross_val_predict(estimator = gb_clf, X = X, y = y, cv = 10)



print('Accuracy :       ', accuracy_score(y, y_pred))
print('ROC :            ', roc_auc_score(label_binarize(y, classes=['A','B']),  
                                         label_binarize(y_pred, classes=['A','B']), average = 'weighted'))
print('F-Measure :      ', f1_score(y, y_pred,  average = 'weighted'))
print('Geometric Mean : ', geometric_mean_score(y, y_pred, average = 'weighted'))
print('Sensitivity :    ', sensitivity_score(y, y_pred, average = 'weighted'))
print('Specificity :    ', specificity_score(y, y_pred, average = 'weighted'))
print('Type I Error :   ', (1-geometric_mean_score(y, y_pred, average = 'weighted')))
print('Type II Error :  ', (1-specificity_score(y, y_pred, average = 'weighted')))

Accuracy :        0.7451913777007763
ROC :             0.6750486374851767
F-Measure :       0.7183398713910999
Geometric Mean :  0.6713945628061343
Sensitivity :     0.7451913777007763
Specificity :     0.6049058972695769
Type I Error :    0.3286054371938657
Type II Error :   0.3950941027304231
